# Serverless Example
## MXNet-Gluon [CLASSIFICATION]

## Setup

In [0]:
# Install some dependencies
!pip install mxnet


import time
import logging
import numpy as np
import pandas as pd
import mxnet as mx
from mxnet import gluon, autograd
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Specify the ConTeXt (CPU or GPU)
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

### Load Database

In [0]:
iris = datasets.load_iris()

X = iris.data
y = iris.target.reshape((-1, 1))

## Training Process

### DataLoaders definition

In [0]:
BATCH_SIZE = 8

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
print('Training examples    : {:5d}'.format(len(X_train)))
print('Validation examples  : {:5d}'.format(len(X_valid)))

train_ds = mx.gluon.data.DataLoader(mx.gluon.data.dataset.ArrayDataset(mx.nd.array(X_train), 
                                                                       mx.nd.array(y_train)),
                                    batch_size=BATCH_SIZE, 
                                    shuffle=True)

valid_ds = mx.gluon.data.DataLoader(mx.gluon.data.dataset.ArrayDataset(mx.nd.array(X_valid), 
                                                                       mx.nd.array(y_valid)),
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False)

Training examples    :   120
Validation examples  :    30


### Model Definition

In [0]:
class Net(gluon.HybridBlock):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            self.fc1 = gluon.nn.Dense(5)
            self.fc2 = gluon.nn.Dense(4)
            self.fc3 = gluon.nn.Dense(3)

    def hybrid_forward(self, F, x):
        # F is a function space that depends on the type of x
        
        # Input size is inferred from the first computing
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # No need of softmax because it is in the loss function
        return self.fc3(x)

In [0]:
model = Net()
model.cast(dtype='float32')
model.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
model.hybridize()

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
optimizer = gluon.Trainer(model.collect_params(), 'sgd', {'learning_rate': 0.01})

### Training
The model will give bad performances due to the lack of normalization of the inputs.

In [0]:
def eval(data_loader, model):
    # Cumulative loss
    cum_loss = 0
    num_examples = 0
    
    for i, (data, label) in enumerate(data_loader):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        num_examples += data.shape[0]
        
        y_pred = model(data)
        loss = softmax_cross_entropy(y_pred, label)
        cum_loss += mx.nd.sum(loss).asscalar()
    return cum_loss/num_examples

In [0]:
EPOCHS = 10
MODEL_PATH = 'model'

train_loss = []    
valid_loss = []

# Time to train
for e in range(EPOCHS):
    tick = time.time()
    
    # Batch training
    for i, (data, label) in enumerate(train_ds):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)

        # Backpropagation
        with autograd.record():
            y_pred = model(data)
            loss = softmax_cross_entropy(y_pred, label)
        loss.backward()
        optimizer.step(data.shape[0])

    # Training metrics
    train_loss.append(eval(train_ds, model))
    
    # Validation metrics
    valid_loss.append(eval(valid_ds, model))
    
    print('Epoch {:3d} [{:.2f} sec] - Training Loss: {:.4f} - Validation Loss: {:.4f}'.format(e+1, time.time()-tick, train_loss[e], valid_loss[e]))

Epoch   1 [0.05 sec] - Training Loss: 1.1146 - Validation Loss: 1.1181
Epoch   2 [0.04 sec] - Training Loss: 1.1003 - Validation Loss: 1.1025
Epoch   3 [0.05 sec] - Training Loss: 1.0981 - Validation Loss: 1.1023
Epoch   4 [0.04 sec] - Training Loss: 1.0977 - Validation Loss: 1.1034
Epoch   5 [0.04 sec] - Training Loss: 1.0973 - Validation Loss: 1.1050
Epoch   6 [0.05 sec] - Training Loss: 1.0971 - Validation Loss: 1.1062
Epoch   7 [0.04 sec] - Training Loss: 1.0967 - Validation Loss: 1.1077
Epoch   8 [0.05 sec] - Training Loss: 1.0965 - Validation Loss: 1.1090
Epoch   9 [0.04 sec] - Training Loss: 1.0963 - Validation Loss: 1.1101
Epoch  10 [0.05 sec] - Training Loss: 1.0961 - Validation Loss: 1.1112


### Save Model

In [0]:
# Save the best model
best_epoch = valid_loss.index(min(valid_loss))
model.export(MODEL_PATH, epoch=best_epoch)
print(f'Best epoch : {best_epoch}')

Best epoch : 2


## Testing Process

### Blank Paper

In [0]:
# Lets put us in blank paper condition
del model

### Prediction

In [0]:
logger = logging.getLogger('iris')


## Prediction 
def handle(event, **kwargs):
    # If data is received as json convert to pandas
    event = event['data'] if 'data' in event else event
    if not isinstance(event, pd.DataFrame):
        event = pd.DataFrame.from_dict(event, orient='columns')

    # Convert to NDArray
    data = mx.nd.array(event.values)
    
    # Retrieve model from disk and use it for predictions
    model = gluon.SymbolBlock.imports(f'{MODEL_PATH}-symbol.json', ['data'], '{}-{:04d}.params'.format(MODEL_PATH, best_epoch))
    
    # Target format convertion
    target_dict = {0: 'setosa', 1: 'versicolor', 2:'virginica'}
    to_target = np.vectorize(lambda x: target_dict[x])
    
    return to_target(np.argmax(model(data).asnumpy(), axis=1)).tolist()

## Testing and liveness check
def test(data, **kwargs):
    pred = handle(data)

    logger.warning(f"predicted: {pred}")
    
    return True


test(iris.data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
predicted: ['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'seto

True